<a href="https://colab.research.google.com/github/LiviaAniely/Aplica-es-do-Processamento-de-Linguagem-Natural/blob/main/L%C3%ADviaA_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinando e praticando Word Embeddings

Neste colab, serão realizadas as seguintes tarefas:
- treinamento de um word embedding em dados jurídicos e avaliação do mesmo em tarefas de similaridade e analogia;
- treinamento de um word embedding em dados de comentários de produtos da Amazon e aplicá-los para classificação de sentimentos.

In [ ]:
from pprint import pprint
from gensim import utils

import numpy as np

from sklearn.model_selection          import train_test_split

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

Nesta atividade você irá exercitar o que aprendeu sobre Word Embeddings na disciplina nas seguintes tarefas:
- treinamento de um word embedding em dados jurídicos e avaliação do mesmo em tarefas de similaridade e analogia;
- treinamento de um word embedding em dados de comentários de produtos da Amazon e aplicá-los para classificação de sentimentos.

# 1. Treinando um word embedding com word2vec

Nesta tarefa iremos criar nossos próprios word embeddings. Para isso usaremos dados que são documentos jurídicos coletados da plataforma Jusbrasil.

Para criar nossos embeddings usaremos a classe `Word2Vec` da biblioteca `gensim`.

In [ ]:
!wget https://raw.githubusercontent.com/issilva5/oclsnippets/master/teor_judiciario.txt -O teor_inteiro_jusbrasil.txt
from gensim.models import Word2Vec

--2023-09-30 01:47:17--  https://raw.githubusercontent.com/issilva5/oclsnippets/master/teor_judiciario.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39955780 (38M) [text/plain]
Saving to: ‘teor_inteiro_jusbrasil.txt’

teor_inteiro_jusbra 100%[===================>]  38.10M   208MB/s    in 0.2s    

2023-09-30 01:47:17 (208 MB/s) - ‘teor_inteiro_jusbrasil.txt’ saved [39955780/39955780]



Essa classe recebe em sua inicialização alguns parâmetros importantes:
 - *corpus_file*: o caminho para um arquivo em que cada documento está contido em uma linha.
 - *vector_size*: o tamanho do vetor de embedding a ser gerado.
 - *window_size*: o tamanho da janela a ser considerada no modelo ao buscar por palavras vizinhas.


 Para a atividade você deve explorar **cinco** valores diferentes para pelo menos um desses parâmetros. Isto é, por exemplo, você pode decidir usar vector_size = 50 e testar variar o window_size entre 1 e 5. Ou window_size = 4 e variar o vector_size em [25, 50, 100, 200, 300].

In [ ]:
# Na forma (vector_size, window_size)

combinacoes = [
    (50,1),
    (50,2),
    (50,3),
    (50,4),
    (50,5),
]

**Treinamento dos modelos:**

In [ ]:
modelos = [Word2Vec(corpus_file='teor_inteiro_jusbrasil.txt', window = ws, vector_size = vs).wv for vs, ws in combinacoes]

## 1.1 Quais as top-5 palavras mais similares a juiz? Como elas variam considerando as diferentes configurações? Você acha que elas fazem sentido?

In [ ]:
for i, modelo in enumerate(modelos):
  print(f'Modelo {i} - vector_size = {combinacoes[i][0]}, window_size = {combinacoes[i][1]}')

  # Cálculo das palavras mais similares
  most_similar_words = modelos[i].most_similar('juiz',topn = 5)

  pprint(most_similar_words)
  print('-'*80)

Modelo 0 - vector_size = 50, window_size = 1
[('magistrado', 0.818531334400177),
 ('juíza', 0.7879314422607422),
 ('desembargador', 0.7224813103675842),
 ('juízo', 0.7134191393852234),
 ('togado', 0.6991901993751526)]
--------------------------------------------------------------------------------
Modelo 1 - vector_size = 50, window_size = 2
[('magistrado', 0.747343897819519),
 ('juíza', 0.7423506379127502),
 ('desembargador', 0.7181423306465149),
 ('julgador', 0.6640152931213379),
 ('togado', 0.6547998785972595)]
--------------------------------------------------------------------------------
Modelo 2 - vector_size = 50, window_size = 3
[('desembargador', 0.7492721676826477),
 ('juíza', 0.7401902675628662),
 ('magistrado', 0.6945540904998779),
 ('(juiz', 0.6548905372619629),
 ('juiz,', 0.635075032711029)]
--------------------------------------------------------------------------------
Modelo 3 - vector_size = 50, window_size = 4
[('juíza', 0.7312962412834167),
 ('desembargador', 0.723

Nos 5 modelos, as respostas retornadas fazem sentido, apesar de algumas variações, são elas: juíza, desembargador, magistrado, juízo, julgador, juiz, togado.

Variando o parâmetro window_size, vemos que as repostas se modificam levemente, de forma a mudar a ordem e o aparecimento de outras palavras, além da própria palavra "juiz" em alguns dos modelos, também é possível observar que em alguns modelos houve repetição de palavras.


## 1.2 Responda a analogia promotora está para juiz como promotor está para o que? Houve diferença nas respostas considerando os diferentes modelos? Qual deu a melhor resposta?

In [ ]:
def analogia(model, x1,x2,y1):
  y2 = model.most_similar(positive = [x2, y1], negative = [x1])
  return y2

for i, modelo in enumerate(modelos):
  print(f'Modelo {i} - vector_size = {combinacoes[i][0]}, window_size = {combinacoes[i][1]}')
  # O cálculo da analogia
  pprint(analogia(modelo, "promotora","juiz","promotor"))

  print('-'*80)

Modelo 0 - vector_size = 50, window_size = 1
[('magistrado', 0.7723170518875122),
 ('juízo', 0.6746059656143188),
 ('juíza', 0.6562816500663757),
 ('juiz,', 0.6557014584541321),
 ('togado', 0.6525790095329285),
 ('desembargador', 0.6475762128829956),
 ('dies', 0.6264380216598511),
 ('apenado', 0.6195945739746094),
 ('julgador', 0.6120823621749878),
 ('indiciado', 0.5898969769477844)]
--------------------------------------------------------------------------------
Modelo 1 - vector_size = 50, window_size = 2
[('magistrado', 0.7505690455436707),
 ('juiz,', 0.7488563656806946),
 ('julgador', 0.6753515005111694),
 ('juízo', 0.6290488839149475),
 ('magistrado,', 0.6214197874069214),
 ('togado', 0.6202765703201294),
 ('desembargador', 0.6173224449157715),
 ('incidente,', 0.5965878963470459),
 ('dies', 0.588655412197113),
 ('fisco', 0.5499539375305176)]
--------------------------------------------------------------------------------
Modelo 2 - vector_size = 50, window_size = 3
[('desembargado

A resposta esperada mais adequeada para a analogia da questão é "juíza".
Observando os modelos, os 5 produziram respostas diferentes, a maioria dos modelos não conseguiu prever a palavra certa também.
Analisando os modelos, vemos que o que obteve melhor posicionamento sobre a analogia foi o modelo 0, que colocou "juíza" na 3° posição.


## 1.3 A variação do parâmetro escolhido impactou na qualidade dos modelos gerados? Porquê você acha que esse parâmetro impactou (ou não) a qualidade dos modelos?

Sim, nota-se que a variação do parâmetro windows_size(window) afetou os resultados, já que é possível observar mudanças nas repostas, seja na verificação das palavras mais similares ou para fazer analogias, e apesar de nesses casos, as respostas no geral não serem tão diferentes, é possível observar alguns problemas em algumas respostas(como repetição de palavras) ou ausência da palavra certa na analogia.

## 1.4 Utilize o modelo que você julgar como o melhor para encontrar um caso de palavra em que as palavras mais similares não fazem muito sentido. Por que você acha que o modelo não foi bem neste caso?

In [ ]:
pprint(modelos[0].most_similar('cumpre',topn = 4))

[('impende', 0.8965938687324524),
 ('convém', 0.894428014755249),
 ('insta', 0.8778426647186279),
 ('devese', 0.8584142327308655)]


Apesar do modelo 0 ter sido o melhor nas tarefas propostas acima, é fato que ele não é perfeito. Como no exemplo acima, ao buscar a palavra mais similar a "cumpre", a palavra "impede" foi a mais similar, definitivamente elas duas não têm um sentido semelhante, o mesmo é válido para as outras palavras retornadas. Como existe um word2vec montado a partir de um texto limitado em quantidade de palavras para analisar, é natural que para certas palavras, não haja um retorno bom. Esse modelo 0 tem o window_size de menor tamanho entre os modelos, esse parâmetro determina quantas palavras de contexto serão usadas para a avaliação, com um número pequeno, é natural que seja levado em consideração apenas as palavras mais próximas, o que para algumas delas, pode promover um resultado não tão bom.

# 2. Usando word embeddings para classificação de sentimento

É possível utilizar word embeddings para classificação de sentimentos através de modelos de linguagem, porém nesta atividade exploraremos uma forma mais simples de usar word embeddings para esta tarefa.

Primeiramente, execute a célula seguinte para gerar o dataset. Os dados que serão utilizados são comentários em produtos no site Amazon.

In [ ]:
!wget https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_neg -O books_pt_neg
!wget https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_pos -O books_pt_pos

corpus_neg = []
corpus_pos = []

for line in open('books_pt_neg'):
  corpus_neg.append(utils.simple_preprocess(line))

for line in open('books_pt_pos'):
  corpus_pos.append(utils.simple_preprocess(line))

--2023-09-30 01:52:24--  https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_neg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 315761 (308K) [text/plain]
Saving to: ‘books_pt_neg’

books_pt_neg        100%[===================>] 308.36K  --.-KB/s    in 0.05s   

2023-09-30 01:52:25 (6.65 MB/s) - ‘books_pt_neg’ saved [315761/315761]

--2023-09-30 01:52:25--  https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

A variável *corpus_neg* contém os comentários negativos, enquanto a variável *corpus_pos* contém os comentários positivos.

In [ ]:
corpus_neg[0]

In [ ]:
corpus_pos[0]

Agora para treinar nossos modelos precisamos obter uma representação de cada sentença como um vetor de features, que chamaremos de *sentence embedding*. Nesta atividade iremos gerar os sentence embedding através da média dos word embeddings das palavras que compõem cada sentença.

Primeiramente, crie um novo modelo Word2Vec usando ambos os conjuntos de sentenças, com parâmetros a sua escolha.

In [ ]:
sentencas = corpus_neg + corpus_pos

modelo = Word2Vec(sentencas, window = 5, vector_size = 50, min_count = 1).wv

Agora vamos criar uma função que dada uma sentença e um modelo Word2Vec retorna o *sentence embedding* dessa sentença. Para calcular a média dos vetores você pode utilizar a função `np.mean`.

In [ ]:
def get_sentence_embedding(sentence, model):

  vec_palavras = []
  sentence_embeddings = []

  for i in range(len(sentence)):
    vec = model[sentence[i]]
    vec_palavras.append(vec)

  sentence_embeddings = np.mean(vec_palavras,axis=0)

  return sentence_embeddings


A seguir você deve aplicar a função sobre os dados.

In [ ]:
# Calcule a lista com os embeddings das sentenças negativas
sentences_embeddings_neg = []

for i in range(len(corpus_neg)):
  if(len(corpus_neg[i]) != 0):
    x = get_sentence_embedding(corpus_neg[i], modelo)
    sentences_embeddings_neg.append(x)

# Calcule a lista com os embeddings das sentenças positivas
sentences_embeddings_pos = []

for i in range(len(corpus_pos)):
  if(len(corpus_pos[i]) != 0):
    x = get_sentence_embedding(corpus_pos[i], modelo)
    sentences_embeddings_pos.append(x)

In [ ]:
X = np.concatenate([sentences_embeddings_neg, sentences_embeddings_pos])
y = np.concatenate([[-1]*len(sentences_embeddings_neg), [1]*len(sentences_embeddings_pos)])

Crie uma partição treino e teste usando a função `train_test_split` do sklearn, usando as variáveis X e y acima.

In [ ]:
# Criação da partição treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)

## 2.1 Agora instancie pelo menos dois modelos de classificação, em seguida os treine e avalie na sua partição. Discuta os resultados em termos das métricas previamentes vistas em sala.

In [ ]:
# Treinamento e avaliação do modelo de regressão logística

clr = LogisticRegression(random_state=0).fit(X_train, y_train)
clr.predict(X_test)[:10]

array([ 1, -1,  1, -1, -1,  1, -1, -1,  1,  1])

In [ ]:
# Métricas
print(classification_report(y_test, clr.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.60      0.53      0.56       204
           1       0.56      0.63      0.60       196

    accuracy                           0.58       400
   macro avg       0.58      0.58      0.58       400
weighted avg       0.58      0.58      0.58       400



In [ ]:
# Treinamento e avaliação do modelo de Perceptron

p = Perceptron(tol=1e-3, random_state=0).fit(X_train, y_train)
p.predict(X_test)[:10]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
# Métricas
print(classification_report(y_test, p.predict(X_test), zero_division = 0.0))

              precision    recall  f1-score   support

          -1       0.51      0.99      0.67       204
           1       0.50      0.01      0.02       196

    accuracy                           0.51       400
   macro avg       0.51      0.50      0.35       400
weighted avg       0.51      0.51      0.35       400



O 1° modelo usado foi o de regressão logística. Analisando as métricas, vê-se que o obteve uma acurácia de 58%.

o 2° modelo usado foi o Perceptron. Ao analisar suas métricas, vê-se uma acurácia de 51%.

O 1° modelo se saiu melhor na classificação, porém ambos modelos, no geral, com esses valores de acurácia não podem ser considerados bons de fato para esses dados.